In [1]:
from datetime import datetime

# date that license deactivation will check against in determinding churn or no-churn,
# usually the date that data storage was created.
pred_datetime = datetime(2020, 7, 21)

# start datetime for license activation.
start_dateime = datetime(2015, 1, 1)

# include both registered and non registered users
registered_only = False

#### this calls another notebook that prepares the data, it combines companies and accounts and then calculates usage across all the metrics. It also calcuates duration

In [3]:
%run ./Churn_Dataframe_JamesMod

In [4]:
df = billable_account_churn_cache
dfWUsage = df.withColumn('UsageGT100', (col('Trips') > 100) | (col('Forms') > 100) | (col('Timekeeping') > 100) | (col('Orders') > 100) )

In [5]:
billable_companies_w_licenses.count()

Out[33]: 9386

In [6]:
%sql

OPTIMIZE delta.`/mnt/silver/cab/timekeeping_statuses`

path,metrics
/mnt/silver/cab/timekeeping_statuses,"List(6, 1961, List(281758, 138889073, 6.9554408E7, 6, 417326451), List(12986, 1068346, 284344.0, 1961, 557600264), 8, null, 1)"


In [7]:
%sql

OPTIMIZE delta.`/mnt/silver/cab/sent_messages`

path,metrics
/mnt/silver/cab/sent_messages,"List(6, 3073, List(97635, 43465019, 1.5528875E7, 6, 93173254), List(3948, 669425, 37109.0, 3073, 114038988), 7, null, 1)"


In [8]:
%sql

OPTIMIZE delta.`/mnt/silver/cab/geofences`

path,metrics
/mnt/silver/cab/geofences,"List(1, 639, List(39293035, 39293035, 3.9293035E7, 1, 39293035), List(3052, 486244, 76201.0, 639, 48693076), 1, null, 1)"


In [9]:
%sql

OPTIMIZE delta.`/mnt/silver/cab/order_headers`

path,metrics
/mnt/silver/cab/order_headers,"List(6, 2307, List(334775, 70941737, 3.6277449E7, 6, 217664696), List(11658, 2740728, 138420.0, 2307, 319336942), 6, null, 1)"


In [10]:
%sql

OPTIMIZE delta.`/mnt/silver/cab/form_headers`

path,metrics
/mnt/silver/cab/form_headers,"List(68, 18415, List(10811, 166405460, 4.6848002E7, 68, 3185664170), List(9685, 3968507, 218035.0, 18415, 4015131516), 72, null, 1)"


In [11]:
%sql

OPTIMIZE delta.`/mnt/silver/cab/trips`

path,metrics
/mnt/silver/cab/trips,"List(79, 26419, List(12676, 880857193, 2.80347374E8, 79, 22147442561), List(11398, 5264172, 1042955.0, 26419, 27553830087), 77, null, 1)"


In [12]:
billable_account_churn.count()

Out[35]: 9386

In [13]:
billable_account_churn_cache.count()

Out[36]: 9386

In [14]:
billable_account_churn.agg({'Duration': 'avg'}).display()

avg(Duration)
402.78681014276583


In [15]:
billable_account_churn_cache.agg({'Duration': 'avg'}).display()

avg(Duration)
402.78681014276583


## overall average duration of a customer

In [17]:
df.agg({'Duration': 'avg'}).display()

avg(Duration)
402.78681014276583


In [18]:
df.filter(col('IsSetupComplete') == True).filter(col('Duration') <= 60).select('Duration').agg({'Duration': 'avg'}).display()

avg(Duration)
27.343601895734597


#### does setup complete affect duration?

In [20]:
df.groupBy('IsSetupComplete').agg({'Duration': 'avg'}).display()

IsSetupComplete,avg(Duration)
true,478.7694934547524
false,357.31624659400546


#### does the number of licenses affect duration?

In [22]:
df.filter(col('Licenses') <= 50 ).agg({'Duration': 'avg'}).display()

avg(Duration)
380.501248365236


#### How about the number of support cases

In [24]:
df.filter((col('Incidents') >= 20)  ).agg({'Duration': 'avg'}).display()

avg(Duration)
852.4551282051282


#### Does the tier change avg duration?

In [26]:
df.groupby('tier').agg({'Duration': 'avg'}).orderBy('tier').display()

tier,avg(Duration)
2,35.875
3,41.375
5,202.41411042944785
10,402.6822848609296
20,423.4134438305709
30,420.547194719472


#### Group by tier and if they had usage then display avg duration

In [28]:
dfWUsage.groupBy('Tier', 'UsageGT100').agg({'Duration': 'avg'}).orderBy('Tier', 'UsageGT100').display()

Tier,UsageGT100,avg(Duration)
2,false,34.142857142857146
2,true,48.0
3,false,41.375
5,false,156.20703125
5,true,371.4
10,false,371.3867028493894
10,true,550.6919786096257
20,false,371.330779054917
20,true,558.003300330033
30,false,377.3269069572506


#### does usage of a certain feature change usage patterns(Forms, timekeeping, orders and trips)

In [30]:
dfWUsage.filter((col('Forms') > 100)).agg({'Duration': 'avg'}).display()

avg(Duration)
683.2056892778993


In [31]:
dfWUsage.filter((col('Timekeeping') > 100)).agg({'Duration': 'avg'}).display()

avg(Duration)
608.6696629213483


In [32]:
dfWUsage.filter((col('Orders') > 100)).agg({'Duration': 'avg'}).display()

avg(Duration)
626.0


In [33]:
dfWUsage.filter((col('Trips') > 100)).agg({'Duration': 'avg'}).display()

avg(Duration)
549.3799201369081


#### does a combination of more than one module change usage

In [35]:
dfWUsage.filter((col('Forms') > 100) & (col('Timekeeping') > 100)).agg({'Duration': 'avg'}).display()

avg(Duration)
712.2127659574468


In [36]:
dfWUsage.filter((col('Forms') > 100) & (col('Orders') > 100)).agg({'Duration': 'avg'}).display()

avg(Duration)
741.4678899082569


In [37]:
dfWUsage.filter((col('Orders') > 100) & (col('Timekeeping') > 100)).agg({'Duration': 'avg'}).display()

avg(Duration)
742.2911392405064


In [38]:
dfWUsage.filter((col('Trips') > 100) & (col('Forms') > 100)).agg({'Duration': 'avg'}).display()

avg(Duration)
697.5166163141994


In [39]:
dfWUsage.filter((col('Trips') > 100) & (col('Timekeeping') > 100)).agg({'Duration': 'avg'}).display()

avg(Duration)
632.5643044619422


In [40]:
dfWUsage.filter((col('Trips') > 100) & (col('Orders') > 100)).agg({'Duration': 'avg'}).display()

avg(Duration)
634.4363636363636


In [41]:
dfWUsage.filter((col('Orders') > 100) & (col('Timekeeping') > 100) & (col('Orders') > 100) & (col('Trips') > 100)).agg({'Duration': 'avg'}).display()

avg(Duration)
739.9027777777778


In [42]:
dfWUsage.filter((col('Orders') > 100) & (col('Timekeeping') > 100)).filter(col('Duration') > 180).count()

Out[34]: 74

In [43]:
dfWUsage.filter((col('Orders') > 100) & (col('Timekeeping') > 100)).filter(col('Duration') > 180).agg({'Duration': 'avg'}).display()

avg(Duration)
789.081081081081


In [44]:
dfWUsage.filter(col('UsageGT100') == True).count()

Out[36]: 1934

In [45]:
dfWUsage.filter(col('Timekeeping') > 100).count()

Out[37]: 445

In [46]:
dfWUsage.filter(col('Timekeeping') > 100).filter(col('Orders') > 100).count()

Out[38]: 79

In [47]:
dfWUsage.filter(col('Timekeeping') > 100).groupBy(col('Tier')).agg({'Duration': 'avg'}).orderBy(col('Tier')).display()

Tier,avg(Duration)
10,594.6320754716982
20,592.0469483568075
30,648.5793650793651


In [48]:
import datetime
from pyspark.sql.functions import year, month, dayofmonth

dfWYear = df.withColumn('Year', year(col('Activation')))

In [49]:
dfWYear.display()

CompanyId,InstanceId,CompanyName,Tier,IsSetupComplete,UserIntegrationType,AccountId,AccountName,City,RegionName,PostalCode,CountryCode,Culture,Activation,Expiration,Deactivation,Duration,Churn,Licenses,Incidents,Trips,Forms,Timekeeping,Orders,Year
822,50,Ward Alternative Energy,10,true,0,694,Ward Alternative Energy,Fort Collins,Colorado,80521,US,en-US,2016-01-21T00:00:00.000+0000,2016-07-22,2016-09-10T13:06:06.760+0000,233,1,9,85,2819,0,66,0,2016
872,50,TECS ELECTRIC LTD,30,true,0,744,TECS ELECTRIC LTD,Weatherford,TX,76087,US,en-US,2016-01-27T00:00:00.000+0000,2016-04-08,2016-09-10T13:06:21.940+0000,227,1,43,0,0,5,4,624,2016
1027,50,Hood River Taxi,10,true,0,899,Hood River Taxi,Hood River,Oregon,97031,US,en-US,2016-02-12T00:00:00.000+0000,2016-04-04,2016-09-10T13:06:52.400+0000,211,1,8,0,922,0,0,0,2016
1107,50,AAA Signs Inc,10,true,0,979,AAA Signs Inc,Sacramento,California,95815,US,en-US,2016-03-04T00:00:00.000+0000,2017-12-21,2017-12-21T00:13:54.140+0000,657,1,34,0,16683,2,0,6,2016
1478,50,Precious Ones Day Care,10,true,0,1340,Precious Ones Day Care,Bethlehem,Pennsylvania,null,US,en-US,2016-04-20T00:00:00.000+0000,2019-06-22,2019-06-22T00:14:20.290+0000,1158,1,5,0,0,0,2,0,2016
1892,50,Pagan Driving School,30,true,0,1754,Pagan Driving School,New York,New York,10032,US,en-US,2015-08-16T00:00:00.000+0000,2016-05-29,2016-09-10T13:10:34.860+0000,391,1,8,0,0,0,0,0,2015
2639,50,Ghbuahn-WFM LEGACY,30,true,0,2500,Ghbuahn-WFM LEGACY,null,null,null,US,en-US,2016-08-08T00:00:00.000+0000,2019-09-09,2019-09-09T00:10:20.640+0000,1127,1,499,65,0,0,0,0,2016
2917,50,LADC,10,true,0,2778,"LADC COMPANIES, INC.",Rogers,Minnesota,55374,US,en-US,2016-03-10T00:00:00.000+0000,2018-09-24,2018-09-27T00:13:03.570+0000,931,1,19,0,1709,0,69,2,2016
2958,50,THE POTTERS HOUSE COMMUNITY,30,false,0,2819,THE POTTERS HOUSE COMMUNITY,null,null,null,US,en-US,2016-04-15T00:00:00.000+0000,2016-04-15,2016-09-10T13:14:52.270+0000,148,1,7,2,0,0,0,0,2016
3094,50,DELAWARE LIFE,30,false,0,2955,DELAWARE LIFE,null,null,null,US,en-US,2016-07-26T00:00:00.000+0000,2017-09-30,2017-09-30T00:01:48.170+0000,431,1,41,0,0,0,0,0,2016


In [50]:
dfWYear.groupBy('year').agg({'Duration': 'avg'}).display()

year,avg(Duration)
2018,378.51028403525953
2015,749.1986301369863
2019,201.8619749045281
2020,84.32874828060523
2016,586.0824175824176
2017,502.311
